# Keras tutorial - the Happy House

Welcome to the first assignment of week 2. In this assignment, you will:
1. Learn to use Keras, a high-level neural networks API (programming framework), written in Python and capable of running on top of several lower-level frameworks including TensorFlow and CNTK. 
2. See how you can in a couple of hours build a deep learning algorithm.

Why are we using Keras? Keras was developed to enable deep learning engineers to build and experiment with different models very quickly. Just as TensorFlow is a higher-level framework than Python, Keras is an even higher-level framework and provides additional abstractions. Being able to go from idea to result with the least possible delay is key to finding good models. However, Keras is more restrictive than the lower-level frameworks, so there are some very complex models that you can implement in TensorFlow but not (without more difficulty) in Keras. That being said, Keras will work fine for many common models. 

In this exercise, you'll work on the "Happy House" problem, which we'll explain below. Let's load the required packages and solve the problem of the Happy House!

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

C:\Users\dell\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 1 - The Happy House 



<img src="images/happy-house.jpg" style="width:350px;height:270px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **the Happy House**</center></caption>


 you are going to build an algorithm which that uses pictures from the front door camera to check if the person is happy or not. The door should open only if the person is happy. 

You have gathered pictures of your friends and yourself, taken by the front-door camera. The dataset is labbeled. 

<img src="images/house-members.png" style="width:550px;height:250px;">

Run the following code to normalize the dataset and learn about its shapes.

In [2]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


**Details of the "Happy" dataset**:
- Images are of shape (64,64,3)
- Training: 600 pictures
- Test: 150 pictures

It is now time to solve the "Happy" Challenge.

## 2 - Building a model in Keras

Keras is very good for rapid prototyping. In just a short time you will be able to build a model that achieves outstanding results.

Here is an example of a model in Keras: 像累积木一样！！

```python
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model
```

除了input，所有中间tensor都用X 命名



In [13]:
# def HappyModel(input_shape):
    
#     X_input = Input(input_shape)

#     # Zero-Padding: pads the border of X_input with zeroes
#     X = ZeroPadding2D((3, 3))(X_input)

#     # CONV -> BN -> RELU Block applied to X
#     X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
#     X = BatchNormalization(axis = 3, name = 'bn0')(X)
#     X = Activation('relu')(X)

#     # MAXPOOL
#     X = MaxPooling2D((2, 2), name='max_pool')(X)

#     # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
#     X = Flatten()(X)
#     X = Dense(1, activation='sigmoid', name='fc')(X)

#     # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
#     model = Model(inputs = X_input, outputs = X, name='HappyModel')

#     return model

In [21]:
#只是组建模型graph！！
def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """

    #input_shape（h,w,c）忽略m 维度  但实际是： (none,input_shape)
    X_input = Input(input_shape)
    #print(X_input.shape)  (?, 64, 64, 3)

    # CONV -> BN -> RELU Block applied to X
    
    #对input image padding，避免边缘信息丢失
    X = ZeroPadding2D((3, 3))(X_input)

    #32个filter 默认是same padding
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0',padding='valid')(X)
   #print(X.shape)#(?, 64, 64, 32)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool0')(X)
    
    
    # CONV -> BN -> RELU Block applied to X
#     X = ZeroPadding2D((3, 3))(X)
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv1',padding='valid')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    #print(X.shape) (?, 18, 18, 64)
    
    X = Flatten()(X)#(?, 18*18*64)
    
    #不能太大，太大准确率低！！
    X = Dense(64, activation='relu', name='fc1')(X)  #, kernel_initializer='he_normal'
    X = Dense(1, activation='sigmoid', name='fc2')(X)

    model = Model(inputs=X_input, outputs=X, name='HappyModel')  
    
    
    return model

### 1 build model graph

In [22]:
#只是组建模型graph
#忽略m维度
happyModel = HappyModel(X_train.shape[1:])

**Exercise**: Implement step 2, i.e. compile the model to configure the learning process. Choose the 3 arguments of `compile()` wisely. Hint: the Happy Challenge is a binary classification problem.

### 2 compile model(指定opt，loss，metrics) 应该是编译成tensorflow

In [23]:
happyModel.compile(optimizer="adam", loss="binary_crossentropy", metrics = ["accuracy"])

### 3  train the model.(指定epochs and the batch size)

In [24]:
happyModel.fit(x=X_train, y=Y_train, epochs=20, batch_size=32, validation_data=(X_test, Y_test))

Train on 600 samples, validate on 150 samples
Epoch 1/20
600/600 [==============================] - 3s 4ms/step - loss: 2.1865 - acc: 0.6767 - val_loss: 0.6169 - val_acc: 0.5733
Epoch 2/20
600/600 [==============================] - 2s 3ms/step - loss: 0.2423 - acc: 0.9200 - val_loss: 0.5730 - val_acc: 0.5733
Epoch 3/20
600/600 [==============================] - 2s 3ms/step - loss: 0.1053 - acc: 0.9667 - val_loss: 0.6127 - val_acc: 0.6333
Epoch 4/20
600/600 [==============================] - 2s 3ms/step - loss: 0.0921 - acc: 0.9717 - val_loss: 0.5185 - val_acc: 0.8067
Epoch 5/20
600/600 [==============================] - 2s 3ms/step - loss: 0.0479 - acc: 0.9817 - val_loss: 0.5130 - val_acc: 0.8000
Epoch 6/20
600/600 [==============================] - 2s 3ms/step - loss: 0.0323 - acc: 0.9917 - val_loss: 0.4886 - val_acc: 0.8267
Epoch 7/20
600/600 [==============================] - 2s 3ms/step - loss: 0.0407 - acc: 0.9900 - val_loss: 0.7212 - val_acc: 0.4933
Epoch 8/20
600/600 [==========

Note that if you run `fit()` again, the `model` will continue to train with the parameters it has already learnt instead of reinitializing them.

**Exercise**: Implement step 4, i.e. test/evaluate the model.

In [ ]:
#preds 包括loss  和 准确率！！！
preds = happyModel.evaluate(x=X_test, y=Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))



If you have not yet achieved a very good accuracy (let's say more than 80%), here're some things you can play around with to try to achieve it:

- Try using blocks of CONV->BATCHNORM->RELU such as:
```python
X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)
```

- You can use MAXPOOL after such blocks. It will help you lower the dimension in height and width.
- Change your optimizer. We find Adam works well. 
- If the model is struggling to run and you get memory issues, lower your batch_size (12 is usually a good compromise)
- Run on more epochs, until you see the train accuracy plateauing. 




## 3 - Conclusion

Congratulations, you have solved the Happy House challenge! 

Now, you just need to link this model to the front-door camera of your house. We unfortunately won't go into the details of how to do that here. 

<font color='red'>
**What we would like you to remember from this assignment:**
- Keras is a tool we recommend for rapid prototyping. It allows you to quickly try out different model architectures. Are there any applications of deep learning to your daily life that you'd like to implement using Keras? 
- Remember how to code a model in Keras and the four steps leading to the evaluation of your model on the test set. Create->Compile->Fit/Train->Evaluate/Test.

## 4 - Test with your own image (Optional)

Congratulations on finishing this assignment. You can now take a picture of your face and see if you could enter the Happy House. To do that:
    1. Click on "File" in the upper bar of this notebook, then click "Open" to go on your Coursera Hub.
    2. Add your image to this Jupyter Notebook's directory, in the "images" folder
    3. Write your image's name in the following code
    4. Run the code and check if the algorithm is right (0 is unhappy, 1 is happy)!
    
The training/test sets were quite similar; for example, all the pictures were taken against the same background (since a front door camera is always mounted in the same position). This makes the problem easier, but a model trained on this data may or may not work on your own data. But feel free to give it a try! 

In [ ]:
### START CODE HERE ###
img_path = 'images/my_image.jpg'
### END CODE HERE ###
img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))

## 5 - Other useful functions in Keras (Optional)

Two other basic features of Keras that you'll find useful are:
- `model.summary()`: prints the details of your layers in a table with the sizes of its inputs/outputs
- `plot_model()`: plots your graph in a nice layout. You can even save it as ".png" using SVG() if you'd like to share it on social media ;). It is saved in "File" then "Open..." in the upper bar of the notebook.

Run the following code.

In [ ]:
happyModel.summary()

In [ ]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))

#因为graphviz需要安装二进制执行文件（跟imagick类似），所以还需要去官网下一个graphviz安装包安装：
#选择第一个msi一路next就可以，然后在环境变量的path里面添加可执行文件的环境变量： bin目录

#没啥意思就是一个流程图